<a href="https://colab.research.google.com/github/avikumart/LLM-GenAI-Notebooks/blob/main/HF_transformers_in_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Transfer learning using Huggingface Transformers and Pytorch

In [1]:
!pip install transformers -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.5 MB/s eta 0:00:00


### BERT embeddings

In [3]:
from transformers import DistilBertTokenizer, DistilBertModel
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained("distilbert-base-uncased")
text = "This is just a sample input to the model"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
output

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BaseModelOutput(last_hidden_state=tensor([[[-0.2530, -0.2836,  0.0833,  ..., -0.0363,  0.0164,  0.6486],
         [-0.3770, -0.5268, -0.0639,  ..., -0.0657,  0.1886,  0.4486],
         [-0.3033, -0.4890,  0.2458,  ...,  0.0120, -0.2598,  0.8252],
         ...,
         [-0.6783, -0.5016,  0.2109,  ...,  0.0414, -0.1782,  0.5918],
         [-0.1551, -0.2563, -0.1981,  ..., -0.2785, -0.2140,  0.2155],
         [ 0.9706,  0.0164, -0.4037,  ...,  0.3323, -0.7224, -0.1955]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)

In [4]:
encoded_input

{'input_ids': tensor([[ 101, 2023, 2003, 2074, 1037, 7099, 7953, 2000, 1996, 2944,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [5]:
output

BaseModelOutput(last_hidden_state=tensor([[[-0.2530, -0.2836,  0.0833,  ..., -0.0363,  0.0164,  0.6486],
         [-0.3770, -0.5268, -0.0639,  ..., -0.0657,  0.1886,  0.4486],
         [-0.3033, -0.4890,  0.2458,  ...,  0.0120, -0.2598,  0.8252],
         ...,
         [-0.6783, -0.5016,  0.2109,  ...,  0.0414, -0.1782,  0.5918],
         [-0.1551, -0.2563, -0.1981,  ..., -0.2785, -0.2140,  0.2155],
         [ 0.9706,  0.0164, -0.4037,  ...,  0.3323, -0.7224, -0.1955]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)

In [7]:
output.last_hidden_state.shape

torch.Size([1, 11, 768])

In [8]:
tokenizer

DistilBertTokenizer(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [9]:
# model without classification head
model

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Li

In [12]:
# distilbert based uncased fine-tuned cola model without classification head
tokenizr = DistilBertTokenizer.from_pretrained('09panesara/distilbert-base-uncased-finetuned-cola')
modl = DistilBertModel.from_pretrained("09panesara/distilbert-base-uncased-finetuned-cola")

Some weights of the model checkpoint at 09panesara/distilbert-base-uncased-finetuned-cola were not used when initializing DistilBertModel: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight', 'classifier.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
txt = "This is just a sample input to the model"
encoded_inpt = tokenizr(txt, return_tensors='pt')
outpt = modl(**encoded_inpt)
outpt

BaseModelOutput(last_hidden_state=tensor([[[ 0.1117, -0.9104, -0.1731,  ...,  0.8763, -0.1914,  1.0186],
         [-0.3203, -0.6272, -0.3526,  ...,  0.5705,  0.0505,  0.5514],
         [-0.0763, -0.5527,  0.0463,  ...,  0.8229, -0.2374,  1.0820],
         ...,
         [-0.3718, -0.6103, -0.0277,  ...,  0.2637, -0.1041,  0.9646],
         [-0.1026, -0.0549, -0.4344,  ..., -0.1853, -0.3288,  0.7640],
         [ 0.9371, -0.0703, -0.3857,  ...,  0.5893, -0.2233, -0.1739]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)

### Classification model

- Using autotokenizer and automodel classes to classify the input text using pre-trained model

In [14]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("09panesara/distilbert-base-uncased-finetuned-cola")

model = AutoModelForSequenceClassification.from_pretrained("09panesara/distilbert-base-uncased-finetuned-cola")


In [19]:
# model with classification head
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [16]:
text = ["It was an amazing movie", "I hated the that product"]
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
output


SequenceClassifierOutput(loss=None, logits=tensor([[-2.9307,  2.7871],
        [ 2.1320, -1.8812]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [18]:
# import numpy and find the prediction # 1: positive, 0: negative
import numpy as np
logits = output.logits.detach().numpy()
y_pred = np.argmax(logits, axis=-1)
y_pred

array([1, 0])